In [ ]:
# default_exp agi

# Chinese BabyAGI

> Using BabyAGI in Chinese to search the Internet

---
* [BabyAGI with LangChain and Faiss](https://python.langchain.com/en/latest/use_cases/agents/baby_agi.html)
* [BabyAGI - LangChain Module](https://python.langchain.com/en/latest/use_cases/autonomous_agents/baby_agi.html)

In [ ]:
# exports
import time
from collections import deque
from typing import Dict, List, Optional, Any

In [ ]:
# exporti
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain

In [ ]:
# exporti
from langchain.embeddings import HuggingFaceEmbeddings

# define your embedding model
embeddings_model = HuggingFaceEmbeddings(model_name='GanymedeNil/text2vec-large-chinese')

In [ ]:
# exporti
import faiss

embedding_size = 1024
# initialize the vectorstore as empty
index = faiss.IndexFlatL2(embedding_size)

In [ ]:
# exporti
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

In [ ]:
# exporti
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [ ]:
# exporti
class TaskCreationChain(LLMChain):
    '''Chain to generates tasks.'''

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        '''Get the response parser.'''
        task_creation_template = (
            '你是一个创建任务的 AI，根据任务的执行结果创建新任务。\n'
            '* 我们的最终目标是「{objective}」\n'
            '* 上一次完成的任务是「{task_description}」\n'
            '  该任务的执行结果为：「\n'
            '{result}\n'
            '」\n'
            '* 这些是未完成的任务：「\n'
            '{incomplete_tasks}\n'
            '」\n\n'
            '请根据上一次完成的任务的结果创建将由 AI 系统完成的新任务。\n'
            '* 请不要创建与未完成的任务重叠的新任务。\n'
            '* 请以带编号的清单形式回复结果，每行只描述一个新任务。\n'
            '  例如：\n'
            '  1. 第一个新任务\n'
            '  2. 第二个新任务\n'
            '* 请从编号 1 开始列出新任务清单。'
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=[
                'objective',
                'task_description',
                'result',
                'incomplete_tasks',
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
# exporti
class TaskPrioritizationChain(LLMChain):
    '''Chain to prioritize tasks.'''

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        '''Get the response parser.'''
        task_prioritization_template = (
            '你是一个根据任务与「最终目标」的相似度给任务的优先级进行排序的 AI。\n'
            '* 我们的最终目标是「{objective}」\n\n'
            '请清理并根据与「最终目标」的相似度从高到低重新排序以下任务：{task_names}。\n'
            '* 请以带编号的清单形式回复结果，每行只描述一个任务。\n'
            '  例如：\n'
            '  {next_task_id}. 第一个任务\n'
            '  {next_next_task_id}. 第二个任务\n'
            '* 请从编号 {next_task_id} 开始列出任务清单。'
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=[
                'objective',
                'task_names',
                'next_task_id',
                'next_next_task_id',
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
# exporti
class ExecutionChain(LLMChain):
    '''Chain to execute tasks.'''

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        '''Get the response parser.'''
        execution_template = (
            '你是一个执行任务的 AI。\n'
            '* 我们的最终目标是「{objective}」\n'
            '* 之前已经完成的任务有：{context}\n\n'
            '请完成这次任务：「{task}」。\n'
            '* 直接回复这次任务的执行结果。'
        )
        prompt = PromptTemplate(
            template = execution_template,
            input_variables = [
                'objective',
                'context',
                'task',
            ],
        )
        return cls(prompt = prompt, llm = llm, verbose = verbose)

In [ ]:
# exporti
def get_next_task(
    task_creation_chain: LLMChain,
    result: Dict,
    task_description: str,
    task_list: List[str],
    objective: str,
) -> List[Dict]:
    '''Get the next task.'''
    incomplete_tasks = '\n'.join(task_list)
    response = task_creation_chain.run(
        objective = objective,
        task_description = task_description,
        result = result,
        incomplete_tasks = incomplete_tasks,
    )
    new_tasks = response.split('\n')
    new_tasks = [task_name.split('.', 1) for task_name in new_tasks]
    new_tasks = [task_name[1].strip() for task_name in new_tasks if len(task_name)==2]
    return [{'task_name': task_name} for task_name in new_tasks if task_name.strip()]

In [ ]:
# exporti
def prioritize_tasks(
    task_prioritization_chain: LLMChain,
    this_task_id: int,
    task_list: List[Dict],
    objective: str,
) -> List[Dict]:
    '''Prioritize tasks.'''
    task_names = [t['task_name'] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(
        objective = objective, task_names = task_names, next_task_id = next_task_id, next_next_task_id = next_task_id + 1,
    )
    new_tasks = response.split('\n')
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split('.', 1)
        if len(task_parts) == 2:
            # task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({'task_id': next_task_id, 'task_name': task_name})
            next_task_id += 1
    return prioritized_task_list

In [ ]:
# exporti
def get_top_tasks(vectorstore, query: str, k: int) -> List[str]:
    '''Get the top k tasks based on the query.'''
    results = vectorstore.similarity_search_with_score(query, k = k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key = lambda x: x[1], reverse = True))
    return [str(item.metadata['task']) for item in sorted_results]


def execute_task(
    vectorstore, execution_chain: LLMChain, objective: str, task: str, k: int = 5
) -> str:
    '''Execute a task.'''
    context = get_top_tasks(vectorstore, query = objective, k = k)
    return execution_chain.run(objective = objective, context = context, task = task)

In [ ]:
# export
class BabyAGI(Chain, BaseModel):
    '''Controller model for the BabyAGI agent.'''

    task_list: deque = Field(default_factory=deque)
    task_creation_chain: TaskCreationChain = Field(...)
    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    execution_chain: Chain = Field(...)
    task_id_counter: int = Field(1)
    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None

    class Config:
        '''Configuration for this pydantic object.'''

        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print('\033[95m\033[1m' + '\n*****TASK LIST*****\n' + '\033[0m\033[0m')
        for t in self.task_list:
            print(str(t['task_id']) + ': ' + t['task_name'])

    def print_next_task(self, task: Dict):
        print('\033[92m\033[1m' + '\n*****NEXT TASK*****\n' + '\033[0m\033[0m')
        print(str(task['task_id']) + ': ' + task['task_name'])

    def print_task_result(self, result: str):
        print('\033[93m\033[1m' + '\n*****TASK RESULT*****\n' + '\033[0m\033[0m')
        print(result)

    @property
    def input_keys(self) -> List[str]:
        return ['objective']

    @property
    def output_keys(self) -> List[str]:
        return []

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        '''Run the agent.'''
        objective = inputs['objective']
        first_task = inputs.get('first_task', f'制作一个最终目标「{objective}」的待办事项清单。')
        num_iters = 0
        while True:
            if self.task_list == deque():
                self.task_id_counter += 1
                self.add_task({'task_id': self.task_id_counter, 'task_name': first_task})

            self.print_task_list()

            # Step 1: Pull the first task
            task = self.task_list.popleft()
            self.print_next_task(task)

            # Step 2: Execute the task
            # todo: while result == ''
            result = execute_task(
                self.vectorstore, self.execution_chain, objective, task['task_name']
            )
            time.sleep(0)
            this_task_id = int(task['task_id'])
            self.print_task_result(result)

            # Step 3: Store the result in Faiss
            result_id = f'result_{task["task_id"]}'
            self.vectorstore.add_texts(
                texts = [result],
                metadatas = [{'task': task['task_name']}],
                ids = [result_id],
            )

            # Step 4: Create new tasks and reprioritize task list
            new_tasks = get_next_task(
                self.task_creation_chain,
                result,
                task['task_name'],
                [t['task_name'] for t in self.task_list],
                objective,
            )
            time.sleep(0)
            for new_task in new_tasks:
                self.task_id_counter += 1
                new_task.update({'task_id': self.task_id_counter})
                self.add_task(new_task)
            self.task_list = deque(
                prioritize_tasks(
                    self.task_prioritization_chain,
                    this_task_id,
                    list(self.task_list),
                    objective,
                )
            )
            time.sleep(0)

            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print(
                    '\033[91m\033[1m' + '\n*****TASK ENDING*****\n' + '\033[0m\033[0m'
                )
                break
        return {}

    @classmethod
    def from_llm(
        cls, llm: BaseLLM, vectorstore: VectorStore,
        task_execution_chain: Optional[Chain] = None,
        verbose: bool = False, **kwargs
    ) -> 'BabyAGI':
        '''Initialize the BabyAGI Controller.'''
        task_creation_chain = TaskCreationChain.from_llm(llm, verbose=verbose)
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        if task_execution_chain is None:
            execution_chain = ExecutionChain.from_llm(llm, verbose=verbose)
        else:
            execution_chain = task_execution_chain
        return cls(
            task_creation_chain = task_creation_chain,
            task_prioritization_chain = task_prioritization_chain,
            execution_chain = execution_chain,
            vectorstore = vectorstore,
            **kwargs,
        )

---
With Tools
* [BabyAGI with Tools - LangChain Module](https://python.langchain.com/en/latest/use_cases/autonomous_agents/baby_agi_with_agent.html)
* [LangChain Utilities](https://github.com/hwchase17/langchain/tree/master/langchain/utilities)

In [ ]:
# export
from langchain import OpenAI, LLMChain
from langchain.agents import Tool
from ipymock.reader import DuckDuckGoSearchAPIWrapper

tools = [
    Tool(
        name = '搜索',
        func = DuckDuckGoSearchAPIWrapper().run,
        description = (
            '如果你需要在互联网上搜索一些你所不确定的最新信息来回答相关问题，请回复：「\n'
            '行动：搜索\n'
            '行动输入：你需要搜索的主题\n'
            '」\n'
            '我将替你在互联网上搜索并给出搜索结果。\n'
            '注意：你的回复中不能包含「行动输出」的字眼，也不能包含「最终结果」的字眼；否则，我会认为你已经确定了「行动输出」或者「最终结果」、也就是已经确定了具体的搜索结果，我就不会再给你提供相应的搜索结果了。'
        ),
        return_direct = True,
    ),
    # Tool(
    #     name = '待办事项',
    #     func = LLMChain(
    #         llm = OpenAI(temperature = 0, verbose = False),
    #         prompt = PromptTemplate.from_template(
    #             '你是一个计划师，擅长为特定目标制定待办清单。\n为以下目标制定一个待办清单：「\n{objective}\n」。'
    #         )
    #     ).run,
    #     description = '适用于需要创建待办事项清单的情况。动作输入：需要创建待办事项清单的最终目标。动作输出：该目标的待办事项清单。请明确指定目标！',
    #     return_direct = True,
    # ),
]

---
Executor
* [LangChain ZeroShotAgent](https://sj-langchain.readthedocs.io/en/latest/modules/agents/examples/custom_agent.html)
  * [Source Code](https://github.com/hwchase17/langchain/blob/master/langchain/agents/mrkl/base.py)
  * [Modular Reasoning, Knowledge and Language system](https://arxiv.org/pdf/2205.00445.pdf)

In [ ]:
# exporti
from langchain.agents import AgentExecutor, ZeroShotAgent

In [ ]:
# export
class ZeroRoundAgent(ZeroShotAgent):
    @property
    def observation_prefix(self) -> str:
        '''Prefix to append the observation with.'''
        return '行动输出：'

    @property
    def llm_prefix(self) -> str:
        '''Prefix to append the llm call with.'''
        return '思考：'

In [ ]:
# export
from langchain import OpenAI
llm = OpenAI(temperature = 0, verbose = False)

In [ ]:
# export
FORMAT_INSTRUCTIONS = '''使用以下格式进行回复：「
任务：你这次必须完成的任务

思考：这次任务和我们的「最终目标」有什么关系？怎么完成这次任务以朝着我们的「最终目标」前进？
行动：这是你所需要采取的行动。请选择 [{tool_names}, ] 中的一个行动。
行动输入：这是与行动相关的输入。
行动输出：
这是行动所导致的输出。
请**避免**编造虚假的「行动输出」。
如果你不确定「行动输出」并且需要我回复与具体的「行动输出」相关的信息来帮助你完成任务，请避免在你的回复中包含「行动输出」的字眼。
如果你的回复中包含「行动输出」的字眼，那么我会认为你已经确定了「行动输出」，我就不会给你提供相应的帮助了。

思考：我现在得到了最终结果／我还不能完全确定最终结果
最终结果：
如果你可以确定已经得到了这次任务的最终的具体结果，请详尽地报告「最终结果」。
请**避免**编造虚假的「最终结果」。如果你不确定「最终结果」，请避免在你的回复中包含「最终结果」的字眼。如果你的回复中包含「最终结果」的字眼，那么我会认为你已经确确实实地完成了这次任务，也就不会和你再交流这次任务。
」'''

llm_chain = LLMChain(
    llm = llm,
    prompt = ZeroRoundAgent.create_prompt(
        tools,
        prefix = '''你是一个执行任务的 AI。\n我们的最终目标是「{objective}」。\n已完成的任务有：{context}。''',
        suffix = '''请完成这次任务：「{task}」\n\n完成这次任务的步骤如下：「\n{agent_scratchpad}''',
        format_instructions = FORMAT_INSTRUCTIONS,
        input_variables = ['objective', 'task', 'context', 'agent_scratchpad'],
    )
)

[LangChain MRKLOutputParser](https://github.com/hwchase17/langchain/blob/master/langchain/agents/mrkl/output_parser.py)

In [ ]:
# export
import re
from typing import Union

from langchain.agents.agent import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish, OutputParserException

FINAL_ANSWER_ACTION = r'最终结果\s*[：|:]\s*(.*)'


class ChineseOutputParser(AgentOutputParser):

    default_action: Optional[str] = None
    default_action_input: Optional[str] = None

    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        match = re.search(FINAL_ANSWER_ACTION, text, re.DOTALL)
        if match:
            return AgentFinish({'output': match.group(1).strip()}, text)
        # \s matches against tab/newline/whitespace
        regex = r'\s*行动\s*\d*\s*[：|:]\s*([^\s]+)\s*行动输入\s*\d*\s*[：|:]\s*([^\n]+)\n*'
        matches = re.findall(regex, text)
        if not matches:
            if self.default_action is not None and self.default_action_input is not None:
                return AgentAction(self.default_action, self.default_action_input, text)
            else:
                return AgentFinish({'output': ''}, text)
            raise OutputParserException(f'Could not parse LLM output: `{text}`')
        action_list = []
        for match in matches[:1]:
            action = match[0].strip()
            if '搜索' in action:
                action = '搜索'
            else:
                continue
            action_input = match[1].strip(' ').strip('"')
            action_list.append(AgentAction(action, action_input, text))
        if action_list == []:
            return AgentFinish({'output': ''}, text)
        return action_list

In [ ]:
# export
agent_executor = AgentExecutor.from_agent_and_tools(
    agent = ZeroRoundAgent(
        llm_chain = llm_chain, allowed_tools = [tool.name for tool in tools], output_parser = ChineseOutputParser()
    ),
    tools = tools, verbose = True, max_iterations = None
)

---
Running the BabyAGI

In [ ]:
# export
# verbose: logging of LLMChains
# max_iterations: Optional[int]
# if None, agi will keep on going forever
baby_agi = BabyAGI.from_llm(
    llm = llm, vectorstore = vectorstore, task_execution_chain = agent_executor, verbose = True, max_iterations = None
)

In [ ]:
# exporti
final_objective = ''

In [ ]:
# exporti
def test_baby_agi(mock_openai):
    baby_agi({'objective': final_objective})

In [ ]:
# exporti
import ipymock
import ipymock.browser

In [ ]:
# export
common = ipymock.browser.common

In [ ]:
# export
def mock_baby_agi(objective):
    global final_objective
    final_objective = objective
    ipymock.do(
        mock_openai = ipymock.browser.mock_openai,
        test_baby_agi = test_baby_agi,
    )